In [9]:
"""
MOID Calculator - Function-based Implementation
Based on Wisniowski & Rickman (2013)
"Fast Geometric Method for Calculating Accurate Minimum Orbit Intersection Distances"

This implementation supports elliptical, parabolic, and hyperbolic orbits.
ENHANCED: Now includes true anomaly outputs at MOID configuration.
"""

import numpy as np
from typing import Tuple, List, Optional, Dict, Union


# Conversion constant
KM_TO_AU = 1.0 / 149597870.7  # km to AU

# ============================================================================
# COORDINATE TRANSFORMATIONS
# ============================================================================

def rotation_matrix_z(angle: float) -> np.ndarray:
    """Rotation matrix about z-axis."""
    c, s = np.cos(angle), np.sin(angle)
    return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])


def rotation_matrix_x(angle: float) -> np.ndarray:
    """Rotation matrix about x-axis."""
    c, s = np.cos(angle), np.sin(angle)
    return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])


def rotate_to_reference_frame(orbit_A: Dict, orbit_B: Dict) -> Tuple[Dict, Dict]:
    """
    Rotate reference frame so orbit A lies in the reference plane.
    This is the preparation step described in Section 2.1 of the paper.
    
    Args:
        orbit_A: Dictionary with keys: a, e, i, omega, Omega
        orbit_B: Dictionary with keys: a, e, i, omega, Omega
        
    Returns:
        Tuple of (rotated_orbit_A, rotated_orbit_B)
    """
    # Orbit A becomes the reference plane
    new_A = {
        'a': orbit_A['a'],
        'e': orbit_A['e'],
        'i': 0.0,
        'omega': 0.0,
        'Omega': 0.0
    }
    
    # Rotation matrices
    R1 = rotation_matrix_z(-orbit_A['Omega'])
    R2 = rotation_matrix_x(-orbit_A['i'])
    R3 = rotation_matrix_z(-orbit_A['omega'])
    R_total = R3 @ R2 @ R1
    
    # Transform orbit B's pole vector
    pole_B = np.array([
        np.sin(orbit_B['Omega']) * np.sin(orbit_B['i']),
        -np.cos(orbit_B['Omega']) * np.sin(orbit_B['i']),
        np.cos(orbit_B['i'])
    ])
    pole_B_new = R_total @ pole_B
    i_new = np.arccos(np.clip(pole_B_new[2], -1, 1))
    
    # Calculate new Omega
    if np.abs(np.sin(i_new)) > 1e-10:
        Omega_new = np.arctan2(pole_B_new[0], -pole_B_new[1])
    else:
        Omega_new = 0.0
    
    # Transform perihelion direction
    peri_dir = np.array([
        np.cos(orbit_B['Omega']) * np.cos(orbit_B['omega']) - 
        np.sin(orbit_B['Omega']) * np.sin(orbit_B['omega']) * np.cos(orbit_B['i']),
        np.sin(orbit_B['Omega']) * np.cos(orbit_B['omega']) + 
        np.cos(orbit_B['Omega']) * np.sin(orbit_B['omega']) * np.cos(orbit_B['i']),
        np.sin(orbit_B['omega']) * np.sin(orbit_B['i'])
    ])
    peri_dir_new = R_total @ peri_dir
    
    # Calculate new omega
    if np.abs(np.sin(i_new)) > 1e-10:
        x_orb = peri_dir_new[0] * np.cos(Omega_new) + peri_dir_new[1] * np.sin(Omega_new)
        y_orb = (-peri_dir_new[0] * np.sin(Omega_new) + peri_dir_new[1] * np.cos(Omega_new)) / np.cos(i_new)
        omega_new = np.arctan2(y_orb, x_orb)
    else:
        omega_new = np.arctan2(peri_dir_new[1], peri_dir_new[0]) - Omega_new
    
    Omega_new = Omega_new % (2 * np.pi)
    omega_new = omega_new % (2 * np.pi)
    
    new_B = {
        'a': orbit_B['a'],
        'e': orbit_B['e'],
        'i': i_new,
        'omega': omega_new,
        'Omega': Omega_new
    }
    
    return new_A, new_B


# ============================================================================
# HELIOCENTRIC DISTANCE CALCULATIONS
# ============================================================================

def heliocentric_distance(a: float, e: float, true_anomaly: float) -> Optional[float]:
    """
    Calculate heliocentric distance for any conic section.
    Uses Eq. (1) from the paper for elliptical orbits.
    
    Args:
        a: Semi-major axis (negative for hyperbola)
        e: Eccentricity
        true_anomaly: True anomaly (radians)
        
    Returns:
        Heliocentric distance, or None if position doesn't exist
    """
    if e < 1.0:
        # Elliptical orbit - Eq. (1)
        return a * (1 - e**2) / (1 + e * np.cos(true_anomaly))
    elif abs(e - 1.0) < 1e-6:
        # Parabolic orbit
        q = a
        return q * (1 + e) / (1 + e * np.cos(true_anomaly))
    else:
        # Hyperbolic orbit
        max_true_anomaly = np.arccos(-1.0 / e)
        if abs(true_anomaly) > max_true_anomaly:
            return None
        return abs(a) * (e**2 - 1) / (1 + e * np.cos(true_anomaly))


def get_scan_range(e: float, hyperbolic_range: float = 6.0) -> Tuple[float, float]:
    """
    Get the scanning range based on orbit type.
    
    Args:
        e: Eccentricity
        hyperbolic_range: Range for hyperbolic orbits (radians)
        
    Returns:
        Tuple of (start_nu, end_nu)
    """
    if e < 1.0:
        return (0.0, 2 * np.pi)
    else:
        return (-hyperbolic_range, hyperbolic_range)


# ============================================================================
# CARTESIAN COORDINATES
# ============================================================================

def cartesian_coords_A(orbit_A: Dict, L: float) -> Optional[np.ndarray]:
    """
    Calculate Cartesian coordinates for object A (in reference plane).
    Object A lies in the reference plane (z=0) as per Section 2.1.
    Uses Eq. (5) for heliocentric distance.
    
    Args:
        orbit_A: Orbital elements dictionary
        L: Longitude (equals true anomaly for reference orbit)
        
    Returns:
        Position vector [x, y, z] or None if invalid
    """
    r = heliocentric_distance(orbit_A['a'], orbit_A['e'], L)
    if r is None:
        return None
    
    x = r * np.cos(L)
    y = r * np.sin(L)
    z = 0.0
    return np.array([x, y, z])


def cartesian_coords_B(orbit_B: Dict, nu: float) -> Optional[np.ndarray]:
    """
    Calculate Cartesian coordinates for object B.
    Uses Eq. (2) from the paper.
    
    Args:
        orbit_B: Orbital elements dictionary
        nu: True anomaly
        
    Returns:
        Position vector [x, y, z] or None if invalid
    """
    r = heliocentric_distance(orbit_B['a'], orbit_B['e'], nu)
    if r is None:
        return None
    
    cos_Om = np.cos(orbit_B['Omega'])
    sin_Om = np.sin(orbit_B['Omega'])
    cos_om_nu = np.cos(orbit_B['omega'] + nu)
    sin_om_nu = np.sin(orbit_B['omega'] + nu)
    cos_i = np.cos(orbit_B['i'])
    sin_i = np.sin(orbit_B['i'])
    
    # Eq. (2)
    x = r * (cos_Om * cos_om_nu - sin_Om * sin_om_nu * cos_i)
    y = r * (sin_Om * cos_om_nu + cos_Om * sin_om_nu * cos_i)
    z = r * sin_om_nu * sin_i
    
    return np.array([x, y, z])


# ============================================================================
# MERIDIONAL PLANE SCANNING (Section 2.2)
# ============================================================================

def meridional_distance(orbit_A: Dict, orbit_B: Dict, nu: float) -> Optional[Tuple[float, float, np.ndarray, np.ndarray]]:
    """
    Calculate meridional distance for a given true anomaly of object B.
    This implements the geometry shown in Figures 1 and 2 of the paper.
    
    The meridional plane is perpendicular to orbit A and contains object B.
    Uses Eq. (3)-(6) from the paper.
    
    Args:
        orbit_A: Orbital elements of object A
        orbit_B: Orbital elements of object B
        nu: True anomaly of object B
        
    Returns:
        Tuple of (distance, longitude_A, pos_A, pos_B) or None if invalid
    """
    # Get position of B - Eq. (2)
    pos_B = cartesian_coords_B(orbit_B, nu)
    if pos_B is None:
        return None
    
    # Calculate longitude L from B's position - Eq. (3), (4)
    rho_B = np.sqrt(pos_B[0]**2 + pos_B[1]**2)
    L = np.arctan2(pos_B[1], pos_B[0])
    
    # Get position of A at this longitude - Eq. (5)
    pos_A = cartesian_coords_A(orbit_A, L)
    if pos_A is None:
        return None
    
    # Calculate meridional distance - Eq. (6)
    # D = sqrt(z_B^2 + (rho_B - r_A)^2)
    r_A = np.linalg.norm(pos_A[:2])
    D = np.sqrt(pos_B[2]**2 + (rho_B - r_A)**2)
    
    return D, L, pos_A, pos_B


def scan_orbits(orbit_A: Dict, orbit_B: Dict, scan_step: float = 0.12, 
                hyperbolic_range: float = 6.0) -> List[Tuple[float, float, float, np.ndarray, np.ndarray]]:
    """
    Scan orbits to find local minima using meridional plane method.
    This implements Section 2.2 of the paper.
    
    The scan_step of 0.12 rad is optimal as stated in the paper:
    "we have found that a scanning step of 0.12 rad is close to optimal"
    
    Args:
        orbit_A: Orbital elements of object A
        orbit_B: Orbital elements of object B
        scan_step: Step size in radians (default 0.12 rad from paper)
        hyperbolic_range: Range for hyperbolic orbits
        
    Returns:
        List of minima as (distance, nu, L, pos_A, pos_B)
    """
    minima = []
    
    # Get scan range based on orbit type
    start_nu, end_nu = get_scan_range(orbit_B['e'], hyperbolic_range)
    
    nu_values = np.arange(start_nu, end_nu, scan_step)
    distances = []
    positions_A = []
    positions_B = []
    L_values = []
    valid_indices = []
    
    # Scan through true anomaly values
    for idx, nu in enumerate(nu_values):
        result = meridional_distance(orbit_A, orbit_B, nu)
        if result is not None:
            D, L, pos_A, pos_B = result
            distances.append(D)
            L_values.append(L)
            positions_A.append(pos_A)
            positions_B.append(pos_B)
            valid_indices.append(idx)
    
    if len(distances) < 3:
        return minima
    
    # Find local minima
    distances = np.array(distances)
    for i in range(1, len(distances) - 1):
        if distances[i] < distances[i-1] and distances[i] < distances[i+1]:
            original_idx = valid_indices[i]
            minima.append((
                distances[i],
                nu_values[original_idx],
                L_values[i],
                positions_A[i],
                positions_B[i]
            ))
    
    # For elliptical orbits, check periodic boundary
    if orbit_B['e'] < 1.0 and len(distances) > 2:
        if distances[0] < distances[1] and distances[0] < distances[-1]:
            minima.append((
                distances[0],
                nu_values[valid_indices[0]],
                L_values[0],
                positions_A[0],
                positions_B[0]
            ))
    
    return minima


# ============================================================================
# PARALLEL TUNING (Section 2.3)
# ============================================================================

def parallel_tune(orbit_A: Dict, orbit_B: Dict, L_init: float, nu_init: float,
                 initial_step: float = 0.06, tune_factor: float = 0.15,
                 target_step: float = 5e-6) -> Tuple[float, float, float]:
    """
    Perform parallel tuning to zoom in on minimum distance.
    This implements the method described in Section 2.3 and illustrated in Figure 3.
    
    The parameters are from the paper:
    - Initial step: 0.06 rad
    - Tune factor: 0.15 (step size multiplier)
    - Target step: 5e-6 rad for initial tuning, 1e-14 rad for final tuning
    
    Args:
        orbit_A: Orbital elements of object A
        orbit_B: Orbital elements of object B
        L_init: Initial longitude for object A
        nu_init: Initial true anomaly for object B
        initial_step: Initial step size (default 0.06 rad from paper)
        tune_factor: Factor to decrease step (default 0.15 from paper)
        target_step: Target step size to reach
        
    Returns:
        Tuple of (final_distance, final_L, final_nu)
    """
    L = L_init
    nu = nu_init
    step = initial_step
    
    while step > target_step:
        improved = True
        while improved:
            improved = False
            
            best_dist_sq = np.inf
            best_L = L
            best_nu = nu
            
            # Test 9 positions (3x3 grid)
            for dL in [-step, 0, step]:
                for dnu in [-step, 0, step]:
                    L_test = L + dL
                    nu_test = nu + dnu
                    
                    pos_A = cartesian_coords_A(orbit_A, L_test)
                    pos_B = cartesian_coords_B(orbit_B, nu_test)
                    
                    if pos_A is None or pos_B is None:
                        continue
                    
                    # Use squared distance to avoid sqrt
                    dist_sq = np.sum((pos_A - pos_B)**2)
                    
                    if dist_sq < best_dist_sq:
                        best_dist_sq = dist_sq
                        best_L = L_test
                        best_nu = nu_test
            
            if best_L != L or best_nu != nu:
                L = best_L
                nu = best_nu
                improved = True
        
        # Decrease step size by tune factor
        step *= tune_factor
    
    # Calculate final distance
    pos_A = cartesian_coords_A(orbit_A, L)
    pos_B = cartesian_coords_B(orbit_B, nu)
    
    if pos_A is None or pos_B is None:
        return np.inf, L, nu
    
    final_dist = np.linalg.norm(pos_A - pos_B)
    return final_dist, L, nu


# ============================================================================
# MAIN MOID CALCULATION (Section 2.4)
# ============================================================================

def calculate_moid(orbit_A: Dict, orbit_B: Dict, 
                  scan_step: float = 0.12,
                  initial_tune_step: float = 0.06,
                  tune_factor: float = 0.15,
                  initial_target_step: float = 5e-6,
                  final_target_step: float = 1e-14,
                  hyperbolic_range: float = 6.0,
                  use_safety: bool = True,
                  return_anomalies: bool = False) -> Union[float, Tuple[float, float, float]]:
    """
    Calculate MOID between two orbits using Wisniowski & Rickman (2013) method.
    
    This implements the complete algorithm described in Section 2.4 of the paper,
    including the safety measures against missing MOIDs.
    
    Parameters from the paper:
    - scan_step: 0.12 rad (optimal as stated in paper)
    - initial_tune_step: 0.06 rad
    - tune_factor: 0.15
    - initial_target_step: 5×10^-6 rad
    - final_target_step: 1×10^-14 rad (double precision limit)
    
    Args:
        orbit_A: Dictionary with keys: a, e, i, omega, Omega
        orbit_B: Dictionary with keys: a, e, i, omega, Omega
        scan_step: Meridional scanning step (default 0.12 rad from paper)
        initial_tune_step: Initial tuning step (default 0.06 rad from paper)
        tune_factor: Step reduction factor (default 0.15 from paper)
        initial_target_step: Initial tuning target (default 5e-6 rad from paper)
        final_target_step: Final tuning target (default 1e-14 rad from paper)
        hyperbolic_range: Scan range for hyperbolic orbits (default 6.0 rad)
        use_safety: Use safety measure for invisible minima (default True)
        return_anomalies: If True, return (MOID, L_A, nu_B) instead of just MOID
        
    Returns:
        If return_anomalies is False:
            MOID in same units as semi-major axis
        If return_anomalies is True:
            Tuple of (MOID, L_A, nu_B) where:
                - MOID: minimum distance in same units as semi-major axis
                - L_A: true anomaly/longitude of object A at MOID (radians)
                - nu_B: true anomaly of object B at MOID (radians)
    """
    # Step 1: Rotate to reference frame (Section 2.1)
    orbit_A_rot, orbit_B_rot = rotate_to_reference_frame(orbit_A, orbit_B)
    
    # Step 2: Scan for minima (Section 2.2)
    minima = scan_orbits(orbit_A_rot, orbit_B_rot, scan_step, hyperbolic_range)
    
    # Step 3: Handle special cases (Section 2.4)
    # Case 1: Only one minimum detected - risk of invisible minimum
    if len(minima) == 1 and use_safety:
        moid_candidates = []
        anomaly_candidates = []
        
        # Use 4 starting points as safety measure
        if orbit_B_rot['e'] >= 1.0:
            max_nu = min(hyperbolic_range,
                        np.arccos(-1.0/orbit_B_rot['e']) - 0.1 if orbit_B_rot['e'] > 1 else hyperbolic_range)
            test_points = np.linspace(-max_nu, max_nu, 4)
        else:
            test_points = [i * np.pi / 2 for i in range(4)]
        
        for nu_start in test_points:
            result = meridional_distance(orbit_A_rot, orbit_B_rot, nu_start)
            if result is None:
                continue
            
            D, L, pos_A, pos_B = result
            
            # Initial tuning
            dist_initial, L_tune, nu_tune = parallel_tune(
                orbit_A_rot, orbit_B_rot, L, nu_start,
                initial_tune_step, tune_factor, initial_target_step
            )
            
            # Final tuning
            dist_final, L_final, nu_final = parallel_tune(
                orbit_A_rot, orbit_B_rot, L_tune, nu_tune,
                initial_tune_step, tune_factor, final_target_step
            )
            
            if dist_final < np.inf:
                moid_candidates.append(dist_final)
                anomaly_candidates.append((L_final, nu_final))
        
        if moid_candidates:
            min_idx = np.argmin(moid_candidates)
            moid = moid_candidates[min_idx]
            L_A, nu_B = anomaly_candidates[min_idx]
            return (moid, L_A, nu_B) if return_anomalies else moid
        else:
            return (np.inf, 0.0, 0.0) if return_anomalies else np.inf
    
    # Case 2: Indistinguishable minima - use safety procedure
    if len(minima) >= 2 and use_safety:
        sorted_minima = sorted(minima, key=lambda x: x[0])
        if len(sorted_minima) >= 2:
            diff = abs(sorted_minima[0][0] - sorted_minima[1][0])
            if diff < 1e-6:
                # Use 4 starting points
                moid_candidates = []
                anomaly_candidates = []
                
                if orbit_B_rot['e'] >= 1.0:
                    max_nu = min(hyperbolic_range,
                                np.arccos(-1.0/orbit_B_rot['e']) - 0.1 if orbit_B_rot['e'] > 1 else hyperbolic_range)
                    test_points = np.linspace(-max_nu, max_nu, 4)
                else:
                    test_points = [i * np.pi / 2 for i in range(4)]
                
                for nu_start in test_points:
                    result = meridional_distance(orbit_A_rot, orbit_B_rot, nu_start)
                    if result is None:
                        continue
                    
                    D, L, pos_A, pos_B = result
                    
                    dist_initial, L_tune, nu_tune = parallel_tune(
                        orbit_A_rot, orbit_B_rot, L, nu_start,
                        initial_tune_step, tune_factor, initial_target_step
                    )
                    
                    dist_final, L_final, nu_final = parallel_tune(
                        orbit_A_rot, orbit_B_rot, L_tune, nu_tune,
                        initial_tune_step, tune_factor, final_target_step
                    )
                    
                    if dist_final < np.inf:
                        moid_candidates.append(dist_final)
                        anomaly_candidates.append((L_final, nu_final))
                
                if moid_candidates:
                    min_idx = np.argmin(moid_candidates)
                    moid = moid_candidates[min_idx]
                    L_A, nu_B = anomaly_candidates[min_idx]
                    return (moid, L_A, nu_B) if return_anomalies else moid
                else:
                    return (np.inf, 0.0, 0.0) if return_anomalies else np.inf
    
    # Case 3: Normal case - tune each detected minimum
    moid_candidates = []
    anomaly_candidates = []
    for D_min, nu, L, pos_A, pos_B in minima:
        # Initial tuning
        dist_initial, L_tune, nu_tune = parallel_tune(
            orbit_A_rot, orbit_B_rot, L, nu,
            initial_tune_step, tune_factor, initial_target_step
        )
        
        # Final tuning
        dist_final, L_final, nu_final = parallel_tune(
            orbit_A_rot, orbit_B_rot, L_tune, nu_tune,
            initial_tune_step, tune_factor, final_target_step
        )
        
        if dist_final < np.inf:
            moid_candidates.append(dist_final)
            anomaly_candidates.append((L_final, nu_final))
    
    # Return the smallest MOID
    if moid_candidates:
        min_idx = np.argmin(moid_candidates)
        moid = moid_candidates[min_idx]
        L_A, nu_B = anomaly_candidates[min_idx]
        return (moid, L_A, nu_B) if return_anomalies else moid
    else:
        # Fallback: brute force search
        start_nu, end_nu = get_scan_range(orbit_B_rot['e'], hyperbolic_range)
        nu_values = np.linspace(start_nu, end_nu, 1000)
        distances = []
        anomalies = []
        for nu in nu_values:
            result = meridional_distance(orbit_A_rot, orbit_B_rot, nu)
            if result is not None:
                distances.append(result[0])
                anomalies.append((result[1], nu))
        
        if distances:
            min_idx = np.argmin(distances)
            moid = distances[min_idx]
            L_A, nu_B = anomalies[min_idx]
            return (moid, L_A, nu_B) if return_anomalies else moid
        else:
            return (np.inf, 0.0, 0.0) if return_anomalies else np.inf


# ============================================================================
# CONVENIENCE FUNCTIONS
# ============================================================================

def create_orbit(a: float, e: float, i: float, omega: float, Omega: float,
                use_degrees: bool = True) -> Dict:
    """
    Create an orbit dictionary.
    
    Args:
        a: Semi-major axis (AU)
        e: Eccentricity
        i: Inclination (degrees or radians)
        omega: Argument of perihelion (degrees or radians)
        Omega: Longitude of ascending node (degrees or radians)
        use_degrees: Whether angles are in degrees (default True)
        
    Returns:
        Dictionary with orbital elements in radians
    """
    if use_degrees:
        i = np.radians(i)
        omega = np.radians(omega)
        Omega = np.radians(Omega)
    
    return {
        'a': a,
        'e': e,
        'i': i,
        'omega': omega,
        'Omega': Omega
    }


def get_orbit_type(e: float) -> str:
    """Return orbit type as string."""
    if e < 1.0:
        return "elliptical"
    elif abs(e - 1.0) < 1e-6:
        return "parabolic"
    else:
        return "hyperbolic"


def get_perihelion_distance(a: float, e: float) -> float:
    """Calculate perihelion distance."""
    if e < 1.0:
        return a * (1 - e)
    else:
        return abs(a) * (e - 1)


# ============================================================================
# EXAMPLE USAGE
# ============================================================================


def test_final_cases():
    """
    Calculate MOIDs for the 4 requested cases:
    1. Earth - Apophis
    2. Earth - 2024 YR4
    3. Apophis - 2024 YR4
    4. Earth - 3I/ATLAS (hyperbolic)
    """
    
    print("\n" + "="*90)
    print("FINAL MOID CALCULATIONS - 4 REQUESTED CASES")
    print("Wisniowski & Rickman (2013) Method - Function-based Implementation")
    print("="*90)
    
    # Define orbital elements (converting from km to AU)
    orbits_data = {
        "Earth": {
            "a": 1.4765067e+08 * KM_TO_AU,
            "e": 9.1669995e-03,
            "i": 4.2422693e-03,
            "omega": 6.64375167e+01,
            "Omega": 1.4760836e+01,
        },
        "Apophis": {
            "a": 1.3793939e+08 * KM_TO_AU,
            "e": 1.9097084e-01,
            "i": 3.3356539e+00,
            "omega": 1.2919949e+02,
            "Omega": 2.0381969e+02,
        },
        "2024_YR4": {
            "a": 3.7680703e+08 * KM_TO_AU,
            "e": 6.6164147e-01,
            "i": 3.4001497e+00,
            "omega": 1.3429905e+02,
            "Omega": 2.7147904e+02,
        },
        "3I_ATLAS": {
            "a": -3.9552667e+07 * KM_TO_AU,  # negative for hyperbola
            "e": 6.1469268e+00,
            "i": 1.7512507e+02,
            "omega": 1.2817255e+02,
            "Omega": 3.2228906e+02,
        },
    }
    
    # Create orbit dictionaries
    orbits = {}
    for name, data in orbits_data.items():
        orbits[name] = create_orbit(
            a=data['a'],
            e=data['e'],
            i=data['i'],
            omega=data['omega'],
            Omega=data['Omega'],
            use_degrees=True
        )
    
    # Display orbital information
    print("\nORBITAL ELEMENTS:")
    print("-"*90)
    for name, orbit in orbits.items():
        orbit_type = get_orbit_type(orbit['e'])
        q = get_perihelion_distance(orbit['a'], orbit['e'])
        
        print(f"\n{name}:")
        print(f"  Orbit type:      {orbit_type}")
        print(f"  Semi-major axis: a = {orbit['a']:12.6f} AU")
        print(f"  Eccentricity:    e = {orbit['e']:12.6f}")
        print(f"  Perihelion:      q = {q:12.6f} AU")
        print(f"  Inclination:     i = {np.degrees(orbit['i']):12.6f}°")
        print(f"  Arg. perihelion: ω = {np.degrees(orbit['omega']):12.6f}°")
        print(f"  Long. asc. node: Ω = {np.degrees(orbit['Omega']):12.6f}°")
    
    # Define the 4 test cases
    test_cases = [
        ("Earth", "Apophis"),
        ("Earth", "2024_YR4"),
        ("Apophis", "2024_YR4"),
        ("Earth", "3I_ATLAS"),
    ]
    
    # Calculate MOIDs
    print("\n" + "="*90)
    print("MOID RESULTS:")
    print("="*90)
    print(f"\n{'Case':<4} {'Object 1':<15} {'Object 2':<15} {'MOID (AU)':<18} {'MOID (km)':<20} {'Status':<20}")
    print("-"*90)
    
    PHA_THRESHOLD = 0.01  # AU
    
    results = []
    for idx, (name1, name2) in enumerate(test_cases, 1):
        orbit1 = orbits[name1]
        orbit2 = orbits[name2]
        
        # Calculate MOID with anomalies
        moid_au, L_A, nu_B = calculate_moid(orbit1, orbit2, return_anomalies=True)
        
        if moid_au < np.inf:
            moid_km = moid_au / KM_TO_AU
            
            # Determine status
            if name1 == "Earth" and moid_au < PHA_THRESHOLD:
                status = "⚠️  POTENTIALLY HAZARDOUS"
            elif name1 == "Earth" and moid_au < 0.20:
                status = "Close approach"
            else:
                status = "Safe"
            
            print(f"{idx:<4} {name1:<15} {name2:<15} {moid_au:<18.10f} {moid_km:<20,.1f} {status:<20}")
            results.append((name1, name2, moid_au, moid_km, status, L_A, nu_B))
        else:
            print(f"{idx:<4} {name1:<15} {name2:<15} {'INFINITE':<18} {'No intersection':<20} {'N/A':<20}")
            results.append((name1, name2, np.inf, np.inf, 'N/A', 0.0, 0.0))
    
    # Detailed analysis
    print("\n" + "="*90)
    print("DETAILED ANALYSIS:")
    print("="*90)
    
    for idx, (name1, name2) in enumerate(test_cases, 1):
        orbit1 = orbits[name1]
        orbit2 = orbits[name2]
        moid_au = results[idx-1][2]
        L_A = results[idx-1][5]
        nu_B = results[idx-1][6]
        
        print(f"\nCase {idx}: {name1} - {name2}")
        print("-"*90)
        
        if moid_au < np.inf:
            moid_km = moid_au / KM_TO_AU
            moid_million_km = moid_km / 1e6
            earth_radii = moid_km / 6371.0  # Earth radius
            
            print(f"  MOID = {moid_au:.10f} AU")
            print(f"  MOID = {moid_km:,.2f} km")
            print(f"  MOID = {moid_million_km:.4f} million km")
            print(f"  MOID = {earth_radii:,.1f} Earth radii")
            
            # NEW: Display anomalies at MOID
            print(f"\n  TRUE ANOMALIES AT MOID CONFIGURATION:")
            print(f"  {name1} true anomaly:  f₁ = {np.degrees(L_A):10.4f}° ({L_A:10.6f} rad)")
            print(f"  {name2} true anomaly:  f₂ = {np.degrees(nu_B):10.4f}° ({nu_B:10.6f} rad)")
            
            if name1 == "Earth":
                if moid_au < PHA_THRESHOLD:
                    print(f"\n  ⚠️  CLASSIFICATION: Potentially Hazardous Asteroid (PHA)")
                    print(f"  ⚠️  MOID is below the {PHA_THRESHOLD} AU threshold")
                else:
                    print(f"\n  ✓ CLASSIFICATION: Not a PHA (MOID > {PHA_THRESHOLD} AU)")
            
            # Additional context for hyperbolic case
            if get_orbit_type(orbit2['e']) == "hyperbolic":
                print(f"\n  Note: {name2} is on a hyperbolic trajectory (interstellar object)")
        else:
            print(f"  No intersection possible (infinite MOID)")
    
    print("\n" + "="*90 + "\n")


if __name__ == "__main__":
    test_final_cases()


FINAL MOID CALCULATIONS - 4 REQUESTED CASES
Wisniowski & Rickman (2013) Method - Function-based Implementation

ORBITAL ELEMENTS:
------------------------------------------------------------------------------------------

Earth:
  Orbit type:      elliptical
  Semi-major axis: a =     0.986984 AU
  Eccentricity:    e =     0.009167
  Perihelion:      q =     0.977936 AU
  Inclination:     i =     0.004242°
  Arg. perihelion: ω =    66.437517°
  Long. asc. node: Ω =    14.760836°

Apophis:
  Orbit type:      elliptical
  Semi-major axis: a =     0.922068 AU
  Eccentricity:    e =     0.190971
  Perihelion:      q =     0.745980 AU
  Inclination:     i =     3.335654°
  Arg. perihelion: ω =   129.199490°
  Long. asc. node: Ω =   203.819690°

2024_YR4:
  Orbit type:      elliptical
  Semi-major axis: a =     2.518799 AU
  Eccentricity:    e =     0.661641
  Perihelion:      q =     0.852257 AU
  Inclination:     i =     3.400150°
  Arg. perihelion: ω =   134.299050°
  Long. asc. node: Ω 

In [10]:
"""
MOID Contour Plot Generator
Creates 2D contour plots showing MOID as a function of true anomalies f1 and f2
"""

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from typing import Dict, Tuple
import sys



def compute_distance_grid(orbit1: Dict, orbit2: Dict, 
                         f1_range: Tuple[float, float],
                         f2_range: Tuple[float, float],
                         n_points: int = 100) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Compute grid of distances for all combinations of true anomalies.
    
    Args:
        orbit1: First orbit dictionary
        orbit2: Second orbit dictionary
        f1_range: (min, max) range for f1 in radians
        f2_range: (min, max) range for f2 in radians
        n_points: Number of grid points in each dimension
        
    Returns:
        Tuple of (f1_grid, f2_grid, distance_grid)
    """
    # Rotate to reference frame
    orbit1_rot, orbit2_rot = rotate_to_reference_frame(orbit1, orbit2)
    
    # Create grid
    f1_values = np.linspace(f1_range[0], f1_range[1], n_points)
    f2_values = np.linspace(f2_range[0], f2_range[1], n_points)
    f1_grid, f2_grid = np.meshgrid(f1_values, f2_values)
    
    # Compute distances
    distance_grid = np.zeros_like(f1_grid)
    
    for i in range(n_points):
        for j in range(n_points):
            f1 = f1_grid[i, j]
            f2 = f2_grid[i, j]
            
            # Get positions
            pos1 = cartesian_coords_A(orbit1_rot, f1)
            pos2 = cartesian_coords_B(orbit2_rot, f2)
            
            if pos1 is not None and pos2 is not None:
                distance_grid[i, j] = np.linalg.norm(pos1 - pos2)
            else:
                distance_grid[i, j] = np.nan
    
    return f1_grid, f2_grid, distance_grid


def plot_moid_contour(orbit1: Dict, orbit2: Dict, 
                     name1: str, name2: str,
                     moid_value: float = None,
                     f1_moid: float = None,
                     f2_moid: float = None,
                     n_points: int = 100,
                     save_filename: str = None):
    """
    Create contour plot of MOID landscape.
    
    Args:
        orbit1: First orbit dictionary
        orbit2: Second orbit dictionary
        name1: Name of first object
        name2: Name of second object
        moid_value: Known MOID value (optional, for marking)
        f1_moid: True anomaly of object 1 at MOID
        f2_moid: True anomaly of object 2 at MOID
        n_points: Grid resolution
        save_filename: Output filename (optional)
    """
    # Determine ranges based on orbit types
    orbit_type1 = get_orbit_type(orbit1['e'])
    orbit_type2 = get_orbit_type(orbit2['e'])
    
    if orbit_type1 == "elliptical":
        f1_range = (0, 2*np.pi)
    else:
        max_f1 = np.arccos(-1.0/orbit1['e']) - 0.1 if orbit1['e'] > 1.0 else 6.0
        f1_range = (-max_f1, max_f1)
    
    if orbit_type2 == "elliptical":
        f2_range = (0, 2*np.pi)
    else:
        max_f2 = np.arccos(-1.0/orbit2['e']) - 0.1 if orbit2['e'] > 1.0 else 6.0
        f2_range = (-max_f2, max_f2)
    
    print(f"\nComputing {n_points}x{n_points} distance grid for {name1} - {name2}...")
    f1_grid, f2_grid, distance_grid = compute_distance_grid(
        orbit1, orbit2, f1_range, f2_range, n_points
    )
    
    # Convert to degrees for plotting
    f1_grid_deg = np.degrees(f1_grid)
    f2_grid_deg = np.degrees(f2_grid)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Find valid (non-NaN) distances
    valid_mask = ~np.isnan(distance_grid)
    if np.any(valid_mask):
        vmin = np.nanmin(distance_grid)
        vmax = np.nanpercentile(distance_grid, 95)  # Use 95th percentile to avoid outliers
    else:
        vmin, vmax = 0, 1
    
    # Create contour plot
    levels = np.linspace(vmin, vmax, 50)
    contour = ax.contourf(f1_grid_deg, f2_grid_deg, distance_grid, 
                          levels=levels, cmap='viridis', extend='max')
    
    # Add contour lines
    contour_lines = ax.contour(f1_grid_deg, f2_grid_deg, distance_grid, 
                               levels=20, colors='white', alpha=0.3, linewidths=0.5)
    
    # Mark the MOID point if provided
    if moid_value is not None and f1_moid is not None and f2_moid is not None:
        f1_moid_deg = np.degrees(f1_moid)
        f2_moid_deg = np.degrees(f2_moid)
        
        ax.plot(f1_moid_deg, f2_moid_deg, 'r*', markersize=20, 
                markeredgecolor='white', markeredgewidth=1.5,
                label=f'MOID = {moid_value:.6f} AU')
        
        # Add crosshairs
        ax.axhline(y=f2_moid_deg, color='red', linestyle='--', alpha=0.5, linewidth=1)
        ax.axvline(x=f1_moid_deg, color='red', linestyle='--', alpha=0.5, linewidth=1)
    
    # Colorbar
    cbar = plt.colorbar(contour, ax=ax, label='Distance (AU)')
    cbar.ax.tick_params(labelsize=10)
    
    # Labels and title
    ax.set_xlabel(f'True Anomaly of {name1}, f₁ (degrees)', fontsize=12, fontweight='bold')
    ax.set_ylabel(f'True Anomaly of {name2}, f₂ (degrees)', fontsize=12, fontweight='bold')
    ax.set_title(f'MOID Contour Plot: {name1} - {name2}\n' + 
                 f'Distance as function of true anomalies',
                 fontsize=14, fontweight='bold', pad=20)
    
    # Grid
    ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.5)
    
    # Legend
    if moid_value is not None:
        ax.legend(loc='upper right', fontsize=11, framealpha=0.9)
    
    # Tight layout
    plt.tight_layout()
    
    # Save or show
    if save_filename:
        plt.savefig(save_filename, dpi=300, bbox_inches='tight')
        print(f"Saved plot to {save_filename}")
    else:
        plt.show()
    
    plt.close()


def create_all_plots():
    """Create contour plots for all 4 test cases."""
    
    print("\n" + "="*80)
    print("GENERATING MOID CONTOUR PLOTS")
    print("="*80)
    
    # Define orbital elements
    orbits_data = {
        "Earth": {
            "a": 1.4765067e+08 * KM_TO_AU,
            "e": 9.1669995e-03,
            "i": 4.2422693e-03,
            "omega": 6.64375167e+01,
            "Omega": 1.4760836e+01,
        },
        "Apophis": {
            "a": 1.3793939e+08 * KM_TO_AU,
            "e": 1.9097084e-01,
            "i": 3.3356539e+00,
            "omega": 1.2919949e+02,
            "Omega": 2.0381969e+02,
        },
        "2024_YR4": {
            "a": 3.7680703e+08 * KM_TO_AU,
            "e": 6.6164147e-01,
            "i": 3.4001497e+00,
            "omega": 1.3429905e+02,
            "Omega": 2.7147904e+02,
        },
        "3I_ATLAS": {
            "a": -3.9552667e+07 * KM_TO_AU,
            "e": 6.1469268e+00,
            "i": 1.7512507e+02,
            "omega": 1.2817255e+02,
            "Omega": 3.2228906e+02,
        },
    }
    
    # Create orbit dictionaries
    orbits = {}
    for name, data in orbits_data.items():
        orbits[name] = create_orbit(
            a=data['a'],
            e=data['e'],
            i=data['i'],
            omega=data['omega'],
            Omega=data['Omega'],
            use_degrees=True
        )
    
    
    # Test cases with known MOID values
    test_cases = [
        ("Earth", "Apophis"),
        ("Earth", "2024_YR4"),
        ("Apophis", "2024_YR4"),
        ("Earth", "3I_ATLAS"),
    ]
    
    for name1, name2 in test_cases:
        print(f"\n{name1} - {name2}")
        print("-"*80)
        
        orbit1 = orbits[name1]
        orbit2 = orbits[name2]
        
        # Calculate MOID with anomalies
        moid_au, f1_moid, f2_moid = calculate_moid(orbit1, orbit2, return_anomalies=True)
        
        if moid_au < np.inf:
            print(f"MOID = {moid_au:.10f} AU")
            print(f"f₁ at MOID = {np.degrees(f1_moid):.4f}°")
            print(f"f₂ at MOID = {np.degrees(f2_moid):.4f}°")
            
            # Create filename
            filename = f"moid_contour_{name1}_{name2}.png"
            
            # Generate plot
            plot_moid_contour(
                orbit1, orbit2, name1, name2,
                moid_value=moid_au,
                f1_moid=f1_moid,
                f2_moid=f2_moid,
                n_points=150,  # Higher resolution
                save_filename=filename
            )
        else:
            print("MOID is infinite - skipping plot")
    
    print("\n" + "="*80)
    print("All plots generated successfully!")
    print("="*80 + "\n")


if __name__ == "__main__":
    create_all_plots()


GENERATING MOID CONTOUR PLOTS

Earth - Apophis
--------------------------------------------------------------------------------
MOID = 0.0056678392 AU
f₁ at MOID = 127.9674°
f₂ at MOID = 236.1549°

Computing 150x150 distance grid for Earth - Apophis...
Saved plot to moid_contour_Earth_Apophis.png

Earth - 2024_YR4
--------------------------------------------------------------------------------
MOID = 0.0017677567 AU
f₁ at MOID = 11.9270°
f₂ at MOID = 47.3502°

Computing 150x150 distance grid for Earth - 2024_YR4...
Saved plot to moid_contour_Earth_2024_YR4.png

Apophis - 2024_YR4
--------------------------------------------------------------------------------
MOID = 0.0503393371 AU
f₁ at MOID = 124.2971°
f₂ at MOID = 51.5060°

Computing 150x150 distance grid for Apophis - 2024_YR4...
Saved plot to moid_contour_Apophis_2024_YR4.png

Earth - 3I_ATLAS
--------------------------------------------------------------------------------
MOID = 0.3784154312 AU
f₁ at MOID = 112.7296°
f₂ at MOID 

In [11]:
def create_lutetia_target():
    """
    Create the target orbit (21) Lutetia as defined in Section 3 of the paper.
    
    From paper Section 3:
    - i = Ω = 0 (reference plane)
    - q = 2.036 AU (perihelion distance)
    - e = 0.164 (eccentricity)
    - ω = 250.227° (argument of perihelion)
    
    Returns:
        Dictionary with orbital elements
    """
    q = 2.036  # AU
    e = 0.164
    a = q / (1 - e)  # Calculate semi-major axis from perihelion distance
    
    return create_orbit(
        a=a,
        e=e,
        i=0.0,
        omega=250.227,
        Omega=0.0,
        use_degrees=True
    )


def run_paper_tests():
    """
    Test all 20 asteroids from Table 1 and compare with Table 2 results.
    
    This validates the MOID calculator against the published results in:
    Wisniowski & Rickman (2013), Table 2
    """
    
    # Table 1: Test asteroid orbital elements
    # Table 2: Expected MOID and Dmin results
    test_asteroids = {
        1: {  # Asteroid (1) Ceres
            'name': '(1) Ceres',
            'category': 'First five asteroids',
            'q': 2.55343183,
            'e': 0.0777898,
            'i': 10.58785,
            'Omega': 80.35052,
            'omega': 72.14554,
            'expected_moid': 0.13455874348909,
            'expected_dmin': 0.1346719
        },
        2: {  # Asteroid (2) Pallas
            'name': '(2) Pallas',
            'category': 'First five asteroids',
            'q': 2.12995319,
            'e': 0.2313469,
            'i': 34.84268,
            'Omega': 173.12520,
            'omega': 310.03850,
            'expected_moid': 0.00289925623680,
            'expected_dmin': 0.0151316
        },
        3: {  # Asteroid (3) Juno
            'name': '(3) Juno',
            'category': 'First five asteroids',
            'q': 1.98948966,
            'e': 0.2552218,
            'i': 12.97943,
            'Omega': 169.90317,
            'omega': 248.22602,
            'expected_moid': 0.07817951779390,
            'expected_dmin': 0.0784614
        },
        4: {  # Asteroid (4) Vesta
            'name': '(4) Vesta',
            'category': 'First five asteroids',
            'q': 2.15354370,
            'e': 0.0882196,
            'i': 7.13426,
            'Omega': 103.89537,
            'omega': 150.08873,
            'expected_moid': 0.08735595371552,
            'expected_dmin': 0.0878071
        },
        5: {  # Asteroid (5) Astraea
            'name': '(5) Astraea',
            'category': 'First five asteroids',
            'q': 2.08388391,
            'e': 0.1905003,
            'i': 5.36719,
            'Omega': 141.60955,
            'omega': 358.80654,
            'expected_moid': 0.14532630925408,
            'expected_dmin': 0.1455354
        },
        6: {  # Asteroid (65407)
            'name': '(65407)',
            'category': 'High eccentricity',
            'q': 2.48391159,
            'e': 0.9543470,
            'i': 119.29902,
            'Omega': 39.00301,
            'omega': 357.90012,
            'expected_moid': 0.26938418933051,
            'expected_dmin': 0.2709789
        },
        7: {  # Asteroid (20461)
            'name': '(20461)',
            'category': 'High eccentricity',
            'q': 2.36382356,
            'e': 0.9006860,
            'i': 160.41316,
            'Omega': 297.34820,
            'omega': 102.45000,
            'expected_moid': 0.54491059333263,
            'expected_dmin': 0.5479893
        },
        8: {  # Asteroid (3200) Phaethon
            'name': '(3200) Phaethon',
            'category': 'High eccentricity',
            'q': 0.13964163,
            'e': 0.8901393,
            'i': 22.23224,
            'Omega': 265.28749,
            'omega': 322.11933,
            'expected_moid': 0.70855959609279,
            'expected_dmin': 0.7152397
        },
        9: {  # Asteroid (2212) Hephaistos
            'name': '(2212) Hephaistos',
            'category': 'High eccentricity',
            'q': 0.35420623,
            'e': 0.8363753,
            'i': 11.68912,
            'Omega': 28.13011,
            'omega': 208.66724,
            'expected_moid': 0.03943927946198,
            'expected_dmin': 0.1120682
        },
        10: {  # Asteroid (4197)
            'name': '(4197)',
            'category': 'High eccentricity',
            'q': 0.52469070,
            'e': 0.7715449,
            'i': 12.56792,
            'Omega': 7.25167,
            'omega': 122.30952,
            'expected_moid': 0.18225709092897,
            'expected_dmin': 0.2183000
        },
        11: {  # Asteroid P5447
            'name': 'P5447',
            'category': 'Low inclination',
            'q': 2.74144856,
            'e': 0.1153501,
            'i': 0.00431,
            'Omega': 272.90217,
            'omega': 251.43828,
            'expected_moid': 0.14766834758223,
            'expected_dmin': 0.1483890
        },
        12: {  # Asteroid U9154
            'name': 'U9154',
            'category': 'Low inclination',
            'q': 2.50571901,
            'e': 0.1924270,
            'i': 0.01522,
            'Omega': 94.14405,
            'omega': 304.71343,
            'expected_moid': 0.00010493251317,
            'expected_dmin': 0.0102467
        },
        13: {  # Asteroid (53910)
            'name': '(53910)',
            'category': 'Low inclination',
            'q': 2.11312640,
            'e': 0.1215091,
            'i': 0.02244,
            'Omega': 321.26045,
            'omega': 109.96758,
            'expected_moid': 0.00030783183432,
            'expected_dmin': 0.0141584
        },
        14: {  # Asteroid G5525
            'name': 'G5525',
            'category': 'Low inclination',
            'q': 2.09876663,
            'e': 0.1543590,
            'i': 0.02731,
            'Omega': 88.64817,
            'omega': 67.91991,
            'expected_moid': 0.00098583168214,
            'expected_dmin': 0.0084652
        },
        15: {  # Asteroid R4450
            'name': 'R4450',
            'category': 'Low inclination',
            'q': 2.67112178,
            'e': 0.1328536,
            'i': 0.02809,
            'Omega': 41.39822,
            'omega': 274.65080,
            'expected_moid': 0.20707625146740,
            'expected_dmin': 0.2087313
        },
        16: {  # Asteroid (61395)
            'name': '(61395)',
            'category': 'Very small MOID',
            'q': 1.99601821,
            'e': 0.1875129,
            'i': 1.26622,
            'Omega': 238.06043,
            'omega': 31.32645,
            'expected_moid': 0.00000003815330,
            'expected_dmin': 0.0069443
        },
        17: {  # Asteroid (64112)
            'name': '(64112)',
            'category': 'Very small MOID',
            'q': 2.03086844,
            'e': 0.1653922,
            'i': 0.66023,
            'Omega': 339.21518,
            'omega': 89.47548,
            'expected_moid': 0.00000419348257,
            'expected_dmin': 0.0011561
        },
        18: {  # Asteroid (27710)
            'name': '(27710)',
            'category': 'Very small MOID',
            'q': 1.77550824,
            'e': 0.1928808,
            'i': 3.43901,
            'Omega': 140.55651,
            'omega': 216.20834,
            'expected_moid': 0.00000627704688,
            'expected_dmin': 0.0402762
        },
        19: {  # Asteroid (61096)
            'name': '(61096)',
            'category': 'Very small MOID',
            'q': 1.96745453,
            'e': 0.1837814,
            'i': 3.69269,
            'Omega': 98.95749,
            'omega': 227.52626,
            'expected_moid': 0.00000785853673,
            'expected_dmin': 0.0043333
        },
        20: {  # Asteroid (56127)
            'name': '(56127)',
            'category': 'Very small MOID',
            'q': 2.15731280,
            'e': 0.1007470,
            'i': 2.91058,
            'Omega': 138.77805,
            'omega': 231.93187,
            'expected_moid': 0.00001189165231,
            'expected_dmin': 0.0100350
        },
    }
    
    # Create Lutetia target orbit
    lutetia = create_lutetia_target()
    
    # Run tests
    print("\n" + "="*110)
    print("VALIDATION AGAINST WISNIOWSKI & RICKMAN (2013) TEST CASES")
    print("Tables 1 and 2 from the paper")
    print("="*110)
    print(f"\nTarget: (21) Lutetia")
    print(f"  Semi-major axis: a = {lutetia['a']:.6f} AU")
    print(f"  Eccentricity:    e = {lutetia['e']}")
    print(f"  Perihelion:      q = {lutetia['a'] * (1 - lutetia['e']):.6f} AU")
    print(f"  Inclination:     i = {np.degrees(lutetia['i']):.6f}°")
    print(f"  Arg. perihelion: ω = {np.degrees(lutetia['omega']):.3f}°")
    print(f"  Long. asc. node: Ω = {np.degrees(lutetia['Omega']):.6f}°")
    
    print("\n" + "-"*110)
    print(f"{'Test':<6} {'Asteroid':<20} {'Category':<22} {'Expected MOID':<18} {'Calculated':<18} {'Diff':<12} {'Status':<10}")
    print("-"*110)
    
    passed = 0
    failed = 0
    tolerance = 1e-8  # 10^-8 AU tolerance (as mentioned in paper Section 2.3)
    results = []
    
    for test_num, data in test_asteroids.items():
        # Calculate semi-major axis from perihelion distance and eccentricity
        a = data['q'] / (1 - data['e'])
        
        # Create asteroid orbit
        asteroid = create_orbit(
            a=a,
            e=data['e'],
            i=data['i'],
            omega=data['omega'],
            Omega=data['Omega'],
            use_degrees=True
        )
        
        # Calculate MOID
        calculated_moid = calculate_moid(lutetia, asteroid)
        expected_moid = data['expected_moid']
        
        # Calculate difference
        diff = abs(calculated_moid - expected_moid)
        
        # Determine status
        if diff < tolerance:
            status = "✓ PASS"
            passed += 1
        else:
            status = "✗ FAIL"
            failed += 1
        
        results.append({
            'test_num': test_num,
            'name': data['name'],
            'category': data['category'],
            'expected': expected_moid,
            'calculated': calculated_moid,
            'diff': diff,
            'status': status
        })
        
        print(f"{test_num:<6} {data['name']:<20} {data['category']:<22} {expected_moid:<18.11f} {calculated_moid:<18.11f} {diff:<12.2e} {status:<10}")
    
    # Summary
    print("-"*110)
    print(f"\nSUMMARY:")
    print(f"  Tests passed: {passed}/{len(test_asteroids)}")
    print(f"  Tests failed: {failed}/{len(test_asteroids)}")
    print(f"  Tolerance:    {tolerance} AU (~{tolerance * 149597870.7:.1f} km)")
    print(f"  Success rate: {passed/len(test_asteroids)*100:.1f}%")
    
    if failed == 0:
        print("\n🎉 ALL TESTS PASSED!")
        print("   Code matches paper results within tolerance.")
        print("   Implementation verified against Wisniowski & Rickman (2013).")
    else:
        print(f"\n⚠️  {failed} test(s) failed.")
        print("   Showing failed tests:")
        for r in results:
            if r['status'] == "✗ FAIL":
                print(f"   - Test {r['test_num']}: {r['name']} (diff = {r['diff']:.2e} AU)")
        
    print("="*110 + "\n")
    
    return passed, failed, results


def detailed_comparison(test_num=16):
    """
    Show detailed comparison for a specific test case.
    
    Test 16 (Asteroid 61395) is particularly challenging as it has a very small 
    MOID of 3.8e-8 AU (~5,700 km), testing the high-precision capabilities.
    
    Args:
        test_num: Test case number (1-20)
    """
    test_asteroids = {
        16: {
            'name': '(61395)',
            'q': 1.99601821,
            'e': 0.1875129,
            'i': 1.26622,
            'Omega': 238.06043,
            'omega': 31.32645,
            'expected_moid': 0.00000003815330,
        }
    }
    
    data = test_asteroids[test_num]
    
    # Create orbits
    lutetia = create_lutetia_target()
    a = data['q'] / (1 - data['e'])
    asteroid = create_orbit(
        a=a,
        e=data['e'],
        i=data['i'],
        omega=data['omega'],
        Omega=data['Omega'],
        use_degrees=True
    )
    
    print(f"\n{'='*90}")
    print(f"DETAILED ANALYSIS: Test Case {test_num} - {data['name']}")
    print(f"{'='*90}")
    print(f"\nAsteroid orbital elements:")
    print(f"  Semi-major axis: a = {a:.8f} AU")
    print(f"  Eccentricity:    e = {data['e']:.8f}")
    print(f"  Inclination:     i = {data['i']:.6f}°")
    print(f"  Long. asc. node: Ω = {data['Omega']:.6f}°")
    print(f"  Arg. perihelion: ω = {data['omega']:.6f}°")
    print(f"  Perihelion dist: q = {data['q']:.8f} AU")
    
    # Calculate MOID
    moid = calculate_moid(lutetia, asteroid)
    
    # Convert to km
    KM_TO_AU = 1.0 / 149597870.7
    expected_km = data['expected_moid'] / KM_TO_AU
    calculated_km = moid / KM_TO_AU
    
    print(f"\nResults:")
    print(f"  Expected MOID:    {data['expected_moid']:.14f} AU  ({expected_km:,.2f} km)")
    print(f"  Calculated MOID:  {moid:.14f} AU  ({calculated_km:,.2f} km)")
    print(f"  Absolute diff:    {abs(moid - data['expected_moid']):.2e} AU")
    print(f"  Relative error:   {abs(moid - data['expected_moid']) / data['expected_moid'] * 100:.6f}%")
    
    # Assessment
    if abs(moid - data['expected_moid']) < 1e-8:
        print(f"\n✓ Result within tolerance (1e-8 AU)")
    else:
        print(f"\n⚠ Result exceeds tolerance (1e-8 AU)")
    
    print(f"{'='*90}\n")


def category_summary(results):
    """
    Provide summary statistics by category of test cases.
    
    Args:
        results: List of test results from run_paper_tests()
    """
    # Group by category
    categories = {}
    for r in results:
        cat = r['category']
        if cat not in categories:
            categories[cat] = {'passed': 0, 'failed': 0, 'max_diff': 0}
        
        if r['status'] == "✓ PASS":
            categories[cat]['passed'] += 1
        else:
            categories[cat]['failed'] += 1
        
        categories[cat]['max_diff'] = max(categories[cat]['max_diff'], r['diff'])
    
    print("\n" + "="*80)
    print("CATEGORY SUMMARY")
    print("="*80)
    print(f"{'Category':<25} {'Passed':<10} {'Failed':<10} {'Total':<10} {'Max Diff (AU)':<15}")
    print("-"*80)
    
    for cat, stats in categories.items():
        total = stats['passed'] + stats['failed']
        print(f"{cat:<25} {stats['passed']:<10} {stats['failed']:<10} {total:<10} {stats['max_diff']:<15.2e}")
    
    print("="*80 + "\n")



if __name__ == "__main__":
    
    # Run all validation tests (only once)
    passed, failed, results = run_paper_tests()
    


VALIDATION AGAINST WISNIOWSKI & RICKMAN (2013) TEST CASES
Tables 1 and 2 from the paper

Target: (21) Lutetia
  Semi-major axis: a = 2.435407 AU
  Eccentricity:    e = 0.164
  Perihelion:      q = 2.036000 AU
  Inclination:     i = 0.000000°
  Arg. perihelion: ω = 250.227°
  Long. asc. node: Ω = 0.000000°

--------------------------------------------------------------------------------------------------------------
Test   Asteroid             Category               Expected MOID      Calculated         Diff         Status    
--------------------------------------------------------------------------------------------------------------
1      (1) Ceres            First five asteroids   0.13455874349      0.13455874619      2.71e-09     ✓ PASS    
2      (2) Pallas           First five asteroids   0.00289925624      0.00289925626      2.60e-11     ✓ PASS    
3      (3) Juno             First five asteroids   0.07817951779      0.07817951807      2.75e-10     ✓ PASS    
4      (4) Vesta 